# Bandit Assignment

This assignment should be done in groups of 2 or 3 and consists of a number of implementation and theory problems based on the topics discussed in the lectures and the course literature (specifically, **version 5** on arXiv):

[Bandits] *Aleksandrs Slivkins, [Introduction to Multi-Armed Bandits](https://arxiv.org/pdf/1904.07272v5.pdf), Found. Trends Mach. Learn. 12(1-2): 1-286 (2019)*

In the implementation problems **(1, 2, 3 and 5)**, you will implement multi-armed bandit algorithms from the [Bandits] book and use them in a provided multi-armed bandit environment. These problems will be graded based on the correctness of the code. You are only required to implement 3 out of 4 of these algorithms and will receive at most 6 points from the implementations. Points may be deducted for incorrect implementation of the 4th algorithm.

In the theory problems **(4 and 6)**, you will derive some properties of the algorithms. These problems will be graded based on the correctness of the arguments.

In the applied problem **(7)**, you will apply the implemented algorithms or another algorithm of your choosing to a bandit problem with irregular real-world data. This problem will be graded based on your algorithm's performance relative to a set of baselines. This problem likely requires you to revise your previously implemented algorithms or implement new algorithms.

You may use the python libraries imported below (*numpy*, *scipy.stats* and *pandas*).

The assignment should be handed in as an updated notebook. The entire notebook should be run before it is handed in, so that the plots are visible. Ensure that it is completely runnable, in the case that we want to reproduce the results. 

## Setup

The cell below contains imports. It may not be modified!

In [ ]:
# DO NOT MODIFY
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import pickle

SEED = 150
ITERATIONS = 20
K = 100
K_CROPS = 3
T = 10000

The cell below contains the bandit environment and may not be modified!

In [ ]:
# DO NOT MODIFY
class Environment:
    def __init__(self, K: int=10, seed: int=0):
        self.random_state = np.random.RandomState(seed=seed)
        self.mu = st.beta.rvs(a=1, b=1, size=K, random_state=self.random_state)
        
    def expected_value(self, a: int) -> float:
        return self.mu[a]
        
    def perform_action(self, a: int) -> int:
        return st.bernoulli.rvs(self.mu[a], random_state=self.random_state)
        
    def optimal_action(self) -> np.intp:
        return np.argmax(self.mu)

class CropEnvironment(Environment):
    def __init__(self, seed: int = 0):
        with open('samples_small.pkl', 'rb') as f:
            samples: np.ndarray = pickle.load(f)
        self.K = samples.shape[0]
        self.N = samples.shape[1]
        samples = samples / samples.max()
        self.random_state = np.random.default_rng(seed=seed)
        self.shuffled_samples = self.random_state.permutation(samples, axis = 0)
        self.mu = np.mean(self.shuffled_samples, axis = 1)

    def expected_value(self, a: int) -> float:
        return self.mu[a]
        
    def perform_action(self, a: int) -> float:
        i = self.random_state.integers(0, self.N)
        return self.shuffled_samples[a,i]
        
    def optimal_action(self) -> int:
        return np.argmax(self.mu)

The cell below contains the bandit algorithm base class and may not be modified!

In [ ]:
# DO NOT MODIFY
class BanditAlgorithmBase:
    def select_action(self) -> int:
        pass
    
    def update(self, action: int, reward: float):
        pass

The cell below contains the bandit experiment and may not be modified!

In [ ]:
# DO NOT MODIFY
class Experiment:
    def __init__(self, environment, bandit_algorithm):
        self.environment = environment
        self.bandit_algorithm = bandit_algorithm
        
    def run_experiment(self, T=100):
        instant_regrets = np.zeros(T)
        for t in range(0, T):
            action = self.bandit_algorithm.select_action()
            reward = self.environment.perform_action(action)
            self.bandit_algorithm.update(action, reward)
            
            optimal_action = self.environment.optimal_action()
            instant_regret = self.environment.expected_value(optimal_action) - self.environment.expected_value(action)
            instant_regrets[t] = instant_regret
        cumulative_regrets = np.cumsum(instant_regrets)
        return (instant_regrets, cumulative_regrets)

The cell below contains a function for repeated experiments with a provided bandit algorithm, averaging regret over the runs. It may not be modified!

In [ ]:
# DO NOT MODIFY
def run_repeated_experiments(bandit_algorithm_class: BanditAlgorithmBase, T: int,  K: int, seed: int, env_class: type = Environment):
    instant_regrets = []
    cumulative_regrets = []
    if env_class == Environment:
        env_constructor = lambda seed: Environment(K, seed)
    elif env_class == CropEnvironment:
        env_constructor = lambda seed: CropEnvironment(seed)
    else:
        raise ValueError(f"Argument env_class must be either 'Environment' or 'CropEnvironment', but got {env_class}.")
    for i in range(ITERATIONS):
        bandit_algorithm = bandit_algorithm_class(T, K)
        environment = env_constructor(seed+i+1)
        experiment = Experiment(environment, bandit_algorithm)

        instant_regrets_i, cumulative_regrets_i = experiment.run_experiment(T)
        instant_regrets.append(instant_regrets_i)
        cumulative_regrets.append(cumulative_regrets_i)
    return pd.DataFrame(data={'t': np.arange(1, T+1),
                             'instant_regret': np.mean(np.vstack(np.array(instant_regrets)), axis=0),
                             'regret': np.mean(np.vstack(np.array(cumulative_regrets)), axis=0)})


## Stochastic Bandits (Chapter 1)

### Problem 1: Implement Explore-First
(2 points, at most 6 points from implementations)

Implement the *Explore-First* algorithm (**Algorithm 1.1** in [Bandits]) within the provided bandit algorithm template below. Use $N = \left(\frac{T}{K}\right)^{2/3} \cdot \left( \log T \right)^{1/3}$.

In [ ]:
class ExploreFirst(BanditAlgorithmBase):
    def __init__(self, T, K):
        """
        Constructor of the bandit algorithm

        Parameters
        ----------
        T : int
            Horizon
        K : int
            Number of actions
        """
        
        # FILL IN CODE HERE
        pass
    
    def select_action(self):
        """
        Select an action which will be performed in the environment in the 
        current time step

        Returns
        -------
        An action index (integer) in [0, K-1]
        """
        
        # FILL IN CODE HERE
        pass
    
    def update(self, action, reward):
        """
        Update the bandit algorithm with the reward received from the 
        environment for the action performed in the current time step

        Parameters
        ----------
        action : int
            An action index (integer) in [0, K-1]
        reward : int
            Reward (integer) in {0, 1} (Bernoulli rewards)

        """
        
        # FILL IN CODE HERE
        pass

Run the algorithm in the provided environment using the code below (averaging the regret over all runs). The exploration and exploitation phases should be clearly visible in the plot.

In [ ]:
# DO NOT MODIFY
np.random.seed(SEED)
ef_df = run_repeated_experiments(ExploreFirst, T, K, SEED)
ef_df.plot(x='t', y='regret', title='Explore-First')

### Problem 2: Implement Epsilon-Greedy
(2 points, at most 6 points from implementations)

Implement the $ \epsilon_t $-*Greedy* algorithm (**Algorithm 1.2** in [Bandits]) within the provided bandit algorithm template below. Use $\epsilon_t = \min \left\{1,\ t^{-1/3} \cdot (K \log t)^{1/3}\right\} $.

In [ ]:
class EpsilonTGreedy(BanditAlgorithmBase):
    def __init__(self, T, K):
        """
        Constructor of the bandit algorithm

        Parameters
        ----------
        T : int
            Horizon
        K : int
            Number of actions
        """
        
        # FILL IN CODE HERE
        pass
    
    def select_action(self):
        """
        Select an action which will be performed in the environment in the 
        current time step

        Returns
        -------
        An action index (integer) in [0, K-1]
        """
        
        # FILL IN CODE HERE
        pass
    
    def update(self, action, reward):
        """
        Update the bandit algorithm with the reward received from the 
        environment for the action performed in the current time step

        Parameters
        ----------
        action : int
            An action index (integer) in [0, K-1]
        reward : int
            Reward (integer) in {0, 1} (Bernoulli rewards)

        """
        
        # FILL IN CODE HERE
        pass

Run the algorithm in the provided environment using the code below (averaging the regret over all runs). The plot should show sublinear regret with respect to $t$.

In [ ]:
# DO NOT MODIFY
np.random.seed(SEED)
eg_df = run_repeated_experiments(EpsilonTGreedy, K, SEED)
eg_df.plot(x='t', y='regret', title='Epsilon_t-Greedy')

### Problem 3: Implement UCB1
(2 points, at most 6 points from implementations)

Implement the UCB1 algorithm (**Algorithm 1.5** in [Bandits]) within the provided bandit algorithm template below.

In [ ]:
class UCB1(BanditAlgorithmBase):
    def __init__(self, T, K):
        """
        Constructor of the bandit algorithm

        Parameters
        ----------
        T : int
            Horizon
        K : int
            Number of actions
        """
        
        # FILL IN CODE HERE
        pass
    
    def select_action(self):
        """
        Select an action which will be performed in the environment in the 
        current time step

        Returns
        -------
        An action index (integer) in [0, K-1]
        """
        
        # FILL IN CODE HERE
        pass
    
    def update(self, action, reward):
        """
        Update the bandit algorithm with the reward received from the 
        environment for the action performed in the current time step

        Parameters
        ----------
        action : int
            An action index (integer) in [0, K-1]
        reward : int
            Reward (integer) in {0, 1} (Bernoulli rewards)

        """
        
        # FILL IN CODE HERE
        pass

Run the algorithm in the provided environment using the code below (averaging the regret over all runs). The plot should show sublinear regret with respect to $t$.

In [ ]:
# DO NOT MODIFY
np.random.seed(SEED)
ucb1_df = run_repeated_experiments(UCB1, T, K, SEED)
ucb1_df.plot(x='t', y='regret', title='UCB1')

### Problem 4: Regret for modified UCB1
(6 points) 

This theory problem is based on **Exercise 1.1** in [Bandits]. The proofs in **Chapter 1** consider environments where the rewards are in the interval $[0,1]$. Consider the case when we have additional knowledge about about the problem and that we know that the rewards for each action are in the interval $\left[\frac{1}{2}, \frac{1}{2} + \epsilon\right]$ for some fixed $\epsilon \in \left(0, \frac{1}{2}\right)$. 

Consider a version of $\text{UCB1}$ modified to utilize this knowledge (you do not need to specify the algorithm completely, just define the new confidence radius $r_t(a)$). For this algorithm and problem setting, prove that:

$\mathbb{E}\left[R(t)\right] \leq \frac{2 \epsilon t}{T^2} + 2 \epsilon \sqrt{2 K t \log T}$

**Instructions:** Use a version of Hoeffding Inequality with ranges (**Theorem A.2** in the [Bandits] book) to modify the confidence radius $r_t(a)$. Subsequently follow the steps of the analysis leading up to **Theorem 1.14** in [Bandits] to derive the regret bound, though show the actual constants instead of using big O notation:

1. Define the clean event, like in **Section 1.3.1**, and lower bound the probability of the event.
2. Start with the definition of the regret $\mathbb{E}\left[R(t)\right]$, and perform a regret decomposition like on **Page 11** of **Section 1.3.2**.
3. Bound the *gap* $\Delta (a_t)$, like in **Section 1.3.3**.
4. Complete the proof using the technique on **Page 12** of **Section 1.3.2**. Note that applying the bound in step 3 requires careful motivation.

*Write the solution in the Markdown cell below (use LaTeX-math mode for equations, etc.).*

#### Solution

WRITE SOLUTION HERE

## Bayesian Bandits (Chapter 3)

### Problem 5: Implement Thompson Sampling
(2 points, at most 6 points from implementations)

Implement the *Thompson Sampling* algorithm (**Algorithm 3.3** in [Bandits]) within the provided bandit algorithm template below. Assume independent priors and that the prior is $\mathbb{P} = \text{Beta}(\alpha_0, \beta_0)$ with $\alpha_0 = 1$ and $\beta_0 = 1$ (i.e. the **Beta-Bernoulli** setting, on **page 35** in [Bandits]).

**Note:** There is a typo in the expression for the posterior $\mathbb{P}_H$ in [Bandits]. It should be $\text{Beta}(\alpha_0 + \text{REW}_H,\ \beta_0 + t - \text{REW}_H)$.

In [ ]:
class ThompsonSampling(BanditAlgorithmBase):
    def __init__(self, T, K):
        """
        Constructor of the bandit algorithm

        Parameters
        ----------
        T : int
            Horizon
        K : int
            Number of actions
        """
        
        # FILL IN CODE HERE
        pass
    
    def select_action(self):
        """
        Select an action which will be performed in the environment in the 
        current time step

        Returns
        -------
        An action index (integer) in [0, K-1]
        """
        
        # FILL IN CODE HERE
        pass
    
    def update(self, action, reward):
        """
        Update the bandit algorithm with the reward received from the 
        environment for the action performed in the current time step

        Parameters
        ----------
        action : int
            An action index (integer) in [0, K-1]
        reward : int
            Reward (integer) in {0, 1} (Bernoulli rewards)

        """
        
        # FILL IN CODE HERE
        pass

Run the algorithm in the provided environment using the code below (averaging the regret over all runs). The plot should show sublinear regret with respect to $t$.

In [ ]:
# DO NOT MODIFY
np.random.seed(SEED)
ts_df = run_repeated_experiments(ThompsonSampling, T, K, SEED)
ts_df.plot(x='t', y='regret', title='Thompson Sampling')

### Problem 6: Regret for Thompson Sampling
(6 points)

In this theory problem, you will show an intermediary step in the proof for the Bayesian regret bound of *Thompson Sampling* in the [Bandits] book.

You are given a $K$-armed bandit problem with rewards in the interval $[0, 1]$. You can assume that $K \leq T$, where $T$ is the horizon. Additionally, you can assume that **Lemma 1.5** holds (i.e., for this assignment we define $r_t (a) := \sqrt{\frac{2  \log T}{ n_t (a)}}$, and then it holds that $\text{Pr}\left\{ \mathcal{E} \right\} \geq 1 - \frac{2}{T^2}$ with $\mathcal{E} := \left\{ \forall a \forall t \;\; \vert \bar{\mu}_t (a) - \mu (a) \vert \leq r_t (a) \right\}$). Then, with $\text{UCB}_t (a) := \bar{\mu}_t (a) + r_t (a)$, show that $\mathbb{E}\left[ \left[ \text{UCB}_t (a) - \mu (a) \right]^{-} \right] \leq \frac{2}{TK}$ (i.e., show that Equation 3.14 in [Bandits], with $\gamma = 2$, holds for all arms $a$ and rounds $t$).

**Note:** $[x]^{-}$ is the negative portion of $x$, i.e., $[x]^{-} = 0$ if $x \geq 0$ and $[x]^{-} = \vert x \vert$ otherwise.

**Hint:** Remember that, given a random variable $X$, an event $\mathcal{E}$ (subset of the sample space) and its complement $\mathcal{E}^c$, by the tower rule, $\mathbb{E}\left[ X \right] = \mathbb{E}\left[ X \;\vert\; \mathcal{E} \right] \cdot \text{Pr}\left\{ \mathcal{E} \right\} + \mathbb{E}\left[ X \;\vert\; \mathcal{E}^c \right] \cdot \text{Pr}\left\{ \mathcal{E}^c \right\}$.

*Write the solution in the Markdown cell below (use LaTeX-math mode for equations, etc.).*

#### Solution

WRITE SOLUTION HERE

## Overall comparison
The cell below plots the regret for all algorithms. Make sure to comment out the any algorithms you have not implemented. 

In [ ]:
fig, ax = plt.subplots(1,1)
fig.suptitle("Final comparison")
ax.set_xlabel("t"); ax.set_ylabel("Regret")
dfs = [ # Comment out any algorithm that you have not implemented.
    (ef_df, "Explore-First"), 
    (eg_df, "Epsilon_t-Greedy"), 
    (ucb1_df, "UCB1"), 
    (ts_df, "Thompson Sampling")
]
for df, label in dfs: df.plot(x='t', y='regret',ax = ax, label = label) 

## Problem 7: Multi-armed bandits for optimizing crop yields

(6 points)

We consider the problem of selecting a date for planting maize grains as a multi-armed bandit problem. Year-to-year crop yields can vary heavily and depend on soil quality, genetics, the weather and other factors. We wish to identify the best date to plant maize among 7 dates. To help us, we will use data from a detailed simulator that generates realistic crop yields.

You have been given samples from $K = 3$ dates to experiment with locally in `samples.pkl`. The crop yields have been rescaled to lie in $[0,1]$ but the distributions do not fit most parametric models, see the histogram of one of the arms below.

![hist](sample_histogram.png?1)

When you submit this notebook to Canvas, your solution will be automatically evaluated on the full dataset of 7 dates. If the evaluation successfully ran, you will receive a comment in Canvas indicating your average total regret. If the evaluation failed to run, you should hopefully receive an error message on your submission. If you do not receive an error message or do not understand the error message, feel to reach out to the responsible TA or visit the consultation sessions. Your submission will be evaluated on 100 runs and must finish under 15 minutes. If your submission exceeds the time limit, you will likely not receive a comment.

The best result for each group will be tracked and uploaded to a [scoreboard on Canvas](https://chalmers.instructure.com/courses/35974/pages/scoreboard) where you will also see the three baselines that you must beat to earn points on this assignment. **Each baseline that you beat yields 2 points, up to a total of 6 points.** In addition, you must describe the process of solving this problem. Your description could include: the algorithms you attempted initially, why you think they worked poorly for this problem, what you tried to improve your algorithm (regardless of success) and a description of the final submission. Descriptions of low quality may reduce your points on this problem.

In order to evaluate your submission, one cell in the notebook must contain the string `SUBMISSION ALGORITHM` on the **first line** and must contain a class declaration that creates a subclass of `BanditAlgorithmBase`, see the example of RandomPolicy below.
**Important to note:** Your class *must not* depend on any external or global variables not provided to it, e.g. the line `self.K = K_CROPS` in `__init__` is not allowed but `self.K = K` is since `K` is a local variable. Similarly, your class *must only* depend on the Python packages provided in the setup cell (changing the setup cell does not affect our evaluation script).

**Hint:** Consider the information you are given about the reward distribution and the assumptions each algorithm assumes of the reward distributions. Which algorithm's assumptions most closely matches (or differs the most from) the information given about the reward distribution?

**Hint:** Some of the algorithms implemented in the previous problems contain hyperparameters that can be tuned to improve performance. Your solutions to previous problems must still match the descriptions in [Bandits]. If you do optimize the hyperparameters, implement a new class below.

**Hint:** The multi-armed bandit literature has plenty of algorithms that could be of interest here, see for example the book by [Lattimore](https://tor-lattimore.com/downloads/book/book.pdf), Successive Elimination (Algorithm 1.4 in [Bandits]), [RandUCB](https://arxiv.org/pdf/1910.04928), [KL-UCB](https://arxiv.org/pdf/1102.2490), Thompson sampling with Gaussian priors, [Feel-Good Thompson sampling](https://arxiv.org/pdf/2110.00871).

### Description of your process and final solution

In [ ]:
# SUBMISSION ALGORITHM
class RandomPolicy(BanditAlgorithmBase): # Feel free to update the name of the policy to better describe your solution.
    def __init__(self, T, K):
        """
        Constructor of the bandit algorithm

        Parameters
        ----------
        T : int
            Horizon
        K : int
            Number of actions
        """
        
        # FILL IN CODE HERE
        self.K = K
    
    def select_action(self):
        """
        Select an action which will be performed in the environment in the 
        current time step

        Returns
        -------
        An action index (integer) in [0, K-1]
        """
        
        # FILL IN CODE HERE
        return np.random.choice(self.K)
    
    def update(self, action, reward):
        """
        Update the bandit algorithm with the reward received from the 
        environment for the action performed in the current time step

        Parameters
        ----------
        action : int
            An action index (integer) in [0, K-1]
        reward : int
            Reward (integer) in {0, 1} (Bernoulli rewards)

        """
        
        # FILL IN CODE HERE
        pass

In [ ]:
np.random.seed(SEED)
policy = RandomPolicy
submission_df = run_repeated_experiments(policy, T, K_CROPS, SEED, CropEnvironment)
submission_df.plot(x = 't', y = 'regret', title = f'Crop Regret ({policy.__name__})')